In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_Operating_System_DATA.pq')

In [4]:
features = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE']).agg(
    {
        'InstallDate00':'max',
        'LastBootUpTime00':'min',
        'TotalVirtualMemorySize00':'max'
    }
).reset_index()
features

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00
0,16783564,2023-01-19,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
1,16783564,2023-01-20,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
2,16783564,2023-01-21,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
3,16783564,2023-01-22,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
4,16783564,2023-01-23,2021-04-02 14:25:05.000,2023-01-23 02:02:49.000,16895
...,...,...,...,...,...
1041725,16818954,2023-06-28,2022-12-01 12:37:51.000,2023-06-27 10:07:20.000,64963
1041726,16818955,2023-06-28,2023-06-27 11:53:37.000,2023-06-27 12:20:35.000,32490
1041727,16818958,2023-06-28,2022-12-01 07:21:03.000,2022-12-01 07:52:38.000,64963
1041728,16818959,2023-06-28,2023-06-27 14:51:45.000,2023-06-27 15:49:41.000,33573


In [7]:
features.to_parquet('../do_not_commit/FeatureDatasets/operating_system_features.pq')


# os_install_date_age


In [24]:
# Age of the operating system calculated from the os_install_age.
# .groupby([MachineID, RWB_effective_date]) and use the .max() value of 'os_install_age'.

# RJ 08/01/23: 'os_install_age' doesn't exist. Using 'InstallDate00'.
os_install_age = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['InstallDate00'].agg('max').reset_index()
os_install_age.to_parquet('../do_not_commit/FeatureDatasets/os_install_age.pq')
os_install_age

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00
0,16783564,2023-01-19,2021-04-02 14:25:05.000
1,16783564,2023-01-20,2021-04-02 14:25:05.000
2,16783564,2023-01-21,2021-04-02 14:25:05.000
3,16783564,2023-01-22,2021-04-02 14:25:05.000
4,16783564,2023-01-23,2021-04-02 14:25:05.000
...,...,...,...
1041725,16818954,2023-06-28,2022-12-01 12:37:51.000
1041726,16818955,2023-06-28,2023-06-27 11:53:37.000
1041727,16818958,2023-06-28,2022-12-01 07:21:03.000
1041728,16818959,2023-06-28,2023-06-27 14:51:45.000



# last_boot_up_time


In [25]:
# Number of days since the machine was last booted.
# Can calculate this from the Operating_System_DATA table using the LastBootUpTime0 attribute.
# .groupby([MachineID, RWB_effective_date]) and use the .min() value of 'LastBootUpTime00'.

last_boot = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['LastBootUpTime00'].agg('min').reset_index()
last_boot.to_parquet('../do_not_commit/FeatureDatasets/last_boot_up_time.pq')
last_boot

,MachineID,RWB_EFFECTIVE_DATE,LastBootUpTime00
0,16783564,2023-01-19,2023-01-16 02:00:46.000
1,16783564,2023-01-20,2023-01-16 02:00:46.000
2,16783564,2023-01-21,2023-01-16 02:00:46.000
3,16783564,2023-01-22,2023-01-16 02:00:46.000
4,16783564,2023-01-23,2023-01-23 02:02:49.000
...,...,...,...
1041725,16818954,2023-06-28,2023-06-27 10:07:20.000
1041726,16818955,2023-06-28,2023-06-27 12:20:35.000
1041727,16818958,2023-06-28,2022-12-01 07:52:38.000
1041728,16818959,2023-06-28,2023-06-27 15:49:41.000



# os_total_virtual_memory_size


In [29]:
# Total virtual operating system memory size.
# .groupby([MachineId, RWB_effective_date]) and use the .max() value of 'TotalVirtualMemorySize00'.

os_total_virtual_memory_size = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['TotalVirtualMemorySize00'].agg('max').reset_index()
os_total_virtual_memory_size.to_parquet('../do_not_commit/FeatureDatasets/os_total_virtual_memory_size.pq')
os_total_virtual_memory_size

,MachineID,RWB_EFFECTIVE_DATE,TotalVirtualMemorySize00
0,16783564,2023-01-19,16895
1,16783564,2023-01-20,16895
2,16783564,2023-01-21,16895
3,16783564,2023-01-22,16895
4,16783564,2023-01-23,16895
...,...,...,...
1041725,16818954,2023-06-28,64963
1041726,16818955,2023-06-28,32490
1041727,16818958,2023-06-28,64963
1041728,16818959,2023-06-28,33573
